LFU 算法

https://labuladong.github.io/algo/di-yi-zhan-da78c/shou-ba-sh-daeca/suan-fa-ji-fb527/

In [2]:
# Least Frequently Used

In [ ]:
# 第一个 freq_table 以频率 freq 为索引，每个索引存放一个双向链表（注意是每个！）
# 这个链表里存放所有使用频率为 freq 的缓存，缓存里存放三个信息，分别为键 key，值 value，以及使用频率 freq

# 第二个 key_table 以键值 key 为索引，每个索引存放对应缓存在 freq_table 中链表里的内存地址

<img src="460.png" style="zoom:50%" />

In [ ]:
# 对于删除操作
# 因为我们保证了链表中从链表头到链表尾的插入时间是有序的
# 所以 freq_table[minFreq] 的链表中链表尾的节点即为使用频率最小，且插入时间最早的节点
# 我们删除它同时根据情况更新 minFreq

<img src="460(2).png" style="zoom:50%" />

In [ ]:
# 460. LFU 缓存
# https://leetcode.cn/problems/lfu-cache/

# 官解
from collections import defaultdict
class Node:
    def __init__(self, key, val, pre=None, nex=None, freq=0):
        self.pre = pre
        self.nex = nex
        self.freq = freq
        self.val = val
        self.key = key
        
    def insert(self, nex):
        nex.pre = self
        nex.nex = self.nex
        self.nex.pre = nex
        self.nex = nex
    
def create_linked_list():
    head = Node(0, 0)
    tail = Node(0, 0)
    head.nex = tail
    tail.pre = head
    return (head, tail)

class LFUCache:
    def __init__(self, capacity: int):
        self.capacity = capacity
        self.size = 0
        self.minFreq = 0
        self.freqMap = defaultdict(create_linked_list)
        self.keyMap = {}

    def delete(self, node):
        if node.pre:
            node.pre.nex = node.nex
            node.nex.pre = node.pre
            if node.pre is self.freqMap[node.freq][0] and node.nex is self.freqMap[node.freq][-1]:
                self.freqMap.pop(node.freq)
        return node.key
        
    def increase(self, node):
        node.freq += 1
        self.delete(node)
        self.freqMap[node.freq][-1].pre.insert(node)
        if node.freq == 1:
            self.minFreq = 1
        elif self.minFreq == node.freq - 1:
            head, tail = self.freqMap[node.freq - 1]
            if head.nex is tail:
                self.minFreq = node.freq

    def get(self, key: int) -> int:
        if key in self.keyMap:
            self.increase(self.keyMap[key])
            return self.keyMap[key].val
        return -1

    def put(self, key: int, value: int) -> None:
        if self.capacity != 0:
            if key in self.keyMap:
                node = self.keyMap[key]
                node.val = value
            else:
                node = Node(key, value)
                self.keyMap[key] = node
                self.size += 1
            if self.size > self.capacity:
                self.size -= 1
                deleted = self.delete(self.freqMap[self.minFreq][0].nex)
                self.keyMap.pop(deleted)
            self.increase(node)


# Your LFUCache object will be instantiated and called as such:
# obj = LFUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

In [ ]:
# 460. LFU 缓存
# https://leetcode.cn/problems/lfu-cache/

# ChatGPT翻译版本
# 17/25
class LFUCache:

    def __init__(self, capacity: int):
        # key 到 val 的映射，我们后文称为 KV 表
        self.keyToVal = {}
        # key 到 freq 的映射，我们后文称为 KF 表
        self.keyToFreq = {}
        # freq 到 key 列表的映射，我们后文称为 FK 表
        self.freqToKeys = {}
        # 记录最小的频次
        self.minFreq = 0
        # 记录 LFU 缓存的最大容量
        self.cap = capacity

    def get(self, key: int) -> int:
        if key not in self.keyToVal:
            return -1
        # 增加 key 对应的 freq
        self.__increaseFreq(key)
        return self.keyToVal[key]

    def put(self, key: int, val: int) -> None:
        if self.cap <= 0:
            return

        # 若 key 已存在，修改对应的 val 即可
        if key in self.keyToVal:
            self.keyToVal[key] = val
            # key 对应的 freq 加一
            self.__increaseFreq(key)
            return

        # key 不存在，需要插入
        # 容量已满的话需要淘汰一个 freq 最小的 key
        if self.cap <= len(self.keyToVal):
            self.__removeMinFreqKey()

        # 插入 key 和 val，对应的 freq 为 1
        # 插入 KV 表
        self.keyToVal[key] = val
        # 插入 KF 表
        self.keyToFreq[key] = 1
        # 插入 FK 表
        self.freqToKeys.setdefault(1, set())
        self.freqToKeys[1].add(key)
        # 插入新 key 后最小的 freq 肯定是 1
        self.minFreq = 1

    def __increaseFreq(self, key: int):
        freq = self.keyToFreq[key]
        # 更新 KF 表
        self.keyToFreq[key] = freq + 1
        # 更新 FK 表
        # 将 key 从 freq 对应的列表中删除
        self.freqToKeys[freq].remove(key)
        # 将 key 加入 freq + 1 对应的列表中
        self.freqToKeys.setdefault(freq + 1, set())
        self.freqToKeys[freq + 1].add(key)
        # 如果 freq 对应的列表空了，移除这个 freq
        if not self.freqToKeys[freq]:
            del self.freqToKeys[freq]
            # 如果这个 freq 恰好是 minFreq，更新 minFreq
            if freq == self.minFreq:
                self.minFreq += 1

    def __removeMinFreqKey(self):
        # freq 最小的 key 列表
        keyList = self.freqToKeys[self.minFreq]
        # 其中最先被插入的那个 key 就是该被淘汰的 key
        deletedKey = keyList.pop()
        # 更新 FK 表
        if not keyList:
            del self.freqToKeys[self.minFreq]
        # 更新 KV 表
        del self.keyToVal[deletedKey]
        # 更新 KF 表
        del self.keyToFreq[deletedKey]

In [ ]:
# setdefault 如果不存在会在原字典里添加一个 key:default_value 并返回 default_value
# get 找不到 key 的时候不会修改原字典，只返回 default_value

In [ ]:
# 首先，self.freqToKeys是一个defaultdict(set)，它是一个特殊的字典
# 当访问不存在的键时，会自动创建一个默认值
# 接下来，.setdefault(1, set())方法会检查键1是否存在于self.freqToKeys字典中
# 如果存在，则直接返回键1对应的值（即一个空的集合）
# 如果不存在，则创建一个默认值为set()的键1
# 最后，.add(key)将key添加到键1对应的集合中，无论是新创建的集合还是已存在的集合
# 综上所述，self.freqToKeys.setdefault(1, set()).add(key)的目的
# 是将key添加到self.freqToKeys字典中键为1的对应集合中
# 如果键1不存在，则会自动创建一个对应的集合并将key添加到集合中

In [ ]:
# 删除某个键key肯定是要同时修改三个映射表的，借助minFreq参数可以从FK表中找到freq最小的keyList
# 根据时序，其中第一个元素就是要被淘汰的deletedKey，操作三个映射表删除这个key即可
# 但是有个细节问题，如果keyList中只有一个元素，那么删除之后minFreq对应的key列表就为空了
# 也就是minFreq变量需要被更新。如何计算当前的minFreq是多少呢？

# 实际上没办法快速计算minFreq，只能线性遍历FK表或者KF表来计算，这样肯定不能保证 O(1) 的时间复杂度
# 但是，其实这里没必要更新minFreq变量，因为你想想removeMinFreqKey这个函数是在什么时候调用？
# 在put方法中插入新key时可能调用。而你回头看put的代码，插入新key时一定会把minFreq更新成 1
# 所以说即便这里minFreq变了，我们也不需要管它

In [ ]:
# 报错用例
["LFUCache","put","put","get","get","put","get","get","get"]
[[2],[2,1],[3,2],[3],[2],[4,3],[2],[3],[4]]

In [ ]:
# https://github.com/labuladong/fucking-algorithm/issues/1113
# 修复刷题全家桶中的多语言解法代码

In [ ]:
from collections import defaultdict
class LFUCache:

    def __init__(self, capacity: int):
        # key 到 val 的映射，我们后文称为 KV 表
        self.keyToVal = {}
        # key 到 freq 的映射，我们后文称为 KF 表
        self.keyToFreq = {}
        # freq 到 key 列表的映射，我们后文称为 FK 表
        self.freqToKeys = defaultdict(list)
        # 记录最小的频次
        self.minFreq = 0
        # 记录 LFU 缓存的最大容量
        self.cap = capacity

    def get(self, key: int) -> int:
        if key not in self.keyToVal:
            return -1
        # 增加 key 对应的 freq
        self.__increaseFreq(key)
        return self.keyToVal[key]

    def put(self, key: int, val: int) -> None:
        if self.cap <= 0:
            return

        # 若 key 已存在，修改对应的 val 即可
        if key in self.keyToVal:
            self.keyToVal[key] = val
            # key 对应的 freq 加一
            self.__increaseFreq(key)
            return

        # key 不存在，需要插入
        # 容量已满的话需要淘汰一个 freq 最小的 key
        if self.cap <= len(self.keyToVal):
            self.__removeMinFreqKey()

        # 插入 key 和 val，对应的 freq 为 1
        # 插入 KV 表
        self.keyToVal[key] = val
        # 插入 KF 表
        self.keyToFreq[key] = 1
        # 插入 FK 表
        self.freqToKeys[1].append(key)
        # 插入新 key 后最小的 freq 肯定是 1
        self.minFreq = 1

    def __increaseFreq(self, key: int):
        freq = self.keyToFreq[key]
        # 更新 KF 表
        self.keyToFreq[key] = freq + 1
        # 更新 FK 表
        # 将 key 从 freq 对应的列表中删除
        self.freqToKeys[freq].remove(key)
        # 将 key 加入 freq + 1 对应的列表中
        self.freqToKeys[freq + 1].append(key)
        # 如果 freq 对应的列表空了，移除这个 freq
        if not self.freqToKeys[freq]:
            del self.freqToKeys[freq]
            # 如果这个 freq 恰好是 minFreq，更新 minFreq
            if freq == self.minFreq:
                self.minFreq += 1

    def __removeMinFreqKey(self):
        # freq 最小的 key 列表
        keyList = self.freqToKeys[self.minFreq]
        # 其中最先被插入的那个 key 就是该被淘汰的 key
        deletedKey = keyList[0]
        del keyList[0]
        # 更新 FK 表
        if not keyList:
            del self.freqToKeys[self.minFreq]
        # 更新 KV 表
        del self.keyToVal[deletedKey]
        # 更新 KF 表
        del self.keyToFreq[deletedKey]

In [ ]:
# 460. LFU 缓存
# https://leetcode.cn/problems/lfu-cache/

# 再写一遍
from collections import defaultdict
class LFUCache:
    def __init__(self, capacity: int):
        self.keyToVal = {}
        self.keyToFreq = {}
        self.freqToKeys = defaultdict(list)
        self.minFreq = 0
        self.capacity = capacity

    def get(self, key: int) -> int:
        if key not in self.keyToVal:
            return -1
        self.__increaseFreq(key)
        return self.keyToVal[key]

    def put(self, key: int, value: int) -> None:
        if key in self.keyToVal:
            self.keyToVal[key] = value
            self.__increaseFreq(key)
            return
        if self.capacity <= len(self.keyToVal):
            self.__removeMinFreqKey()
        self.keyToVal[key] = value
        self.keyToFreq[key] = 1
        self.freqToKeys[1].append(key)
        self.minFreq = 1

    def __increaseFreq(self, key: int):
        freq = self.keyToFreq[key]
        self.keyToFreq[key] = freq + 1
        self.freqToKeys[freq].remove(key)
        self.freqToKeys[freq + 1].append(key)
        if not self.freqToKeys[freq]:
            del self.freqToKeys[freq]
            if freq == self.minFreq:
                self.minFreq += 1

    def __removeMinFreqKey(self):
        keyList = self.freqToKeys[self.minFreq]
        deletedKey = keyList[0]
        del keyList[0]
        if not keyList:
            del self.freqToKeys[self.minFreq]
        del self.keyToVal[deletedKey]
        del self.keyToFreq[deletedKey]


# Your LFUCache object will be instantiated and called as such:
# obj = LFUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)

In [ ]:
# 460. LFU 缓存
# https://leetcode.cn/problems/lfu-cache/

# 再写一遍
class LFUCache:
    def __init__(self, capacity: int):
        self.keyToVal = {}
        self.keyToFreq = {}
        self.freqToKey = defaultdict(list)
        self.capacity = capacity
        self.minFreq = 0

    def get(self, key: int) -> int:
        if key not in self.keyToVal:
            return -1
        self.__increaseFreq(key)
        return self.keyToVal[key]

    def put(self, key: int, value: int) -> None:
        pass

    def __increaseFreq(self, key):
        pass


# Your LFUCache object will be instantiated and called as such:
# obj = LFUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)